In [18]:
import requests #DBSCCAN
from bs4 import BeautifulSoup
import re

import pandas as pd

#get Indonesian District Cordinate by scrapping
def getIndonesiaCityCordinates():
    arrayOfDistrict=[]
    arrayOfLatitude=[]
    arrayOfLongitude=[]
    URL = 'https://www.latlong.net/category/cities-103-15.html'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    tr_list = soup.find_all('tr')
    for tr in tr_list:
        td_list = tr.find_all('td')
        if(len(td_list)>0):
            sep = ','
            district = td_list[0].text.split(sep, 1)[0].lower().replace('city ', '')
            latitude = td_list[1].text
            longitude = td_list[2].text
            if (district not in arrayOfDistrict):
                arrayOfDistrict.append(district)
                arrayOfLatitude.append(latitude)
                arrayOfLongitude.append(longitude)
    df_location = pd.DataFrame()
    d  = {'district': arrayOfDistrict, 'latitude': arrayOfLatitude, 'longitude': arrayOfLongitude,}
    df_location = pd.DataFrame(d, columns=['district','latitude', 'longitude'])
    return df_location

df_location = getIndonesiaCityCordinates();
df_location.to_csv('output/df_location.csv', index=None)

In [4]:
#function
#scrapping to detik.com 
def getDetikNewsUrls(keyword):
    urls = []
    keyword = keyword.replace(' ', '+')
    URL = 'https://www.detik.com/search/searchall?query='+keyword+'&siteid=2'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    articles = soup.find_all('article')
    for article in articles:
        for link in article.find_all('a'):
            urls.append(link.get('href'))
    return urls

In [5]:
#function
#get paragraph content, only with number is collected

import requests
from bs4 import BeautifulSoup
import re

diseases = ['demam berdarah', 'malaria', 'diare', 'tuberkulosis']
districts = ['bengkayang' , 'kotawaringin barat' , 'kotawaringin timur' , 'kapuas' , 'barito selatan' , 'barito utara' , 'sukamara' , 'lamandau' , 'seruyan' , 'katingan' , 'pulang pisau' , 'gunung mas' , 'palangka raya' , 'tanah laut' , 'baru' , 'barito kuala' , 'tapin' , 'hulu sungai selatan' , 'hulu sungai tengah' , 'hulu sungai utara' , 'tabalong' , 'tanah bumbu' , 'balangan' , 'banjarmasin' , 'banjar baru' , 'pasir' , 'kutai barat' , 'kutai' , 'kutai timur' , 'berau' , 'malinau' , 'penajam paser utara' , 'balikpapan' , 'samarinda' , 'tarakan' , 'bontang' , 'minahasa selatan' , 'bolaang mongondow utara' , 'manado' , 'bitung' , 'kotamobagu' , 'banggai' , 'morowali' , 'poso' , 'donggala' , 'tojo una-una' , 'palu' , 'selayar' , 'bulukumba' , 'bantaeng' , 'jeneponto' , 'takalar' , 'gowa' , 'sinjai' , 'maros' , 'pangkajene kepulauan' , 'barru' , 'bone' , 'soppeng' , 'wajo' , 'sidenreng rappang' , 'pinrang' , 'enrekang' , 'luwu' , 'tana toraja' , 'luwu utara' , 'luwu timur' , 'makassar' , 'pare-pare' , 'palopo' , 'buton' , 'kolaka' , 'konawe selatan' , 'bombana' , 'wakatobi' , 'kolaka utara' , 'buton utara' , 'konawe utara' , 'kendari' , 'bau-bau' , 'gorontalo' , 'polewali mandar' , 'mamuju' , 'mamuju utara' , 'maluku tenggara' , 'maluku tengah' , 'seram bagian barat' , 'seram bagian timur' , 'ambon' , 'halmahera barat' , 'halmahera utara' , 'ternate' , 'tidore kepulauan' , 'fakfak' , 'teluk bintuni' , 'manokwari' , 'sorong' , 'nabire' , 'mimika' , 'jayapura']


def getUrlContentParagraph(url):
    paragraphs = []
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        ps = soup.find_all('p')
        count=0
        for p in ps:
            if ('img' not in str(p) and 'Gambas' not in str(p) and 'Selanjutnya' not in str(p)):
                paragraphs.append(p.text)
    except:
        pass
    return paragraphs

def getDetailDate(url):
    detail_date=None
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        detail_dates = soup.findAll("div", {"class": "detail__date"})
        if (len(detail_dates)>0):
            detail_date = detail_dates[0].text
    except:
        pass
    return detail_date

In [6]:
#news to paragraph
newsUrls = []

newsParagraphs = []

#prepare array for dataframe
arrayKeyword = []
arrayNewsUrl = []
arrayNewsParagraph = []
arrayDate = []
arrayDisease = []

for disease in diseases:
    for district in districts:
        urls= getDetikNewsUrls(disease+" "+district)
        for url in urls:
            paragraphs = getUrlContentParagraph(url)
            detail_date = getDetailDate(url)
            for paragraph in paragraphs:
                arrayDate.append(detail_date)
                arrayKeyword.append(disease+" "+district)
                arrayNewsUrl.append(url)
                arrayNewsParagraph.append(paragraph)
                arrayDisease.append(disease)

In [7]:
import pandas as pd

d = {'date':arrayDate, 'keyword':arrayKeyword, 'url':arrayNewsUrl, 'paragraph': arrayNewsParagraph, 'category': arrayDisease}
df = pd.DataFrame(d, columns=['date','keyword', 'url', 'paragraph', 'category'])
df = df[df['paragraph'] != ''] #remove blank paragraph
df.reset_index(inplace = True, drop=True) #reset index

In [8]:
#to csv
df.to_csv('output/df3.csv', index=None)

In [9]:
df 

,date,keyword,url,paragraph,category
0,"Sabtu, 04 Apr 2020 17:22 WIB",demam berdarah palangka raya,https://news.detik.com/berita/d-4965300/pernya...,Pemerintah memperbarui data peningkatan kasus ...,demam berdarah
1,"Sabtu, 04 Apr 2020 17:22 WIB",demam berdarah palangka raya,https://news.detik.com/berita/d-4965300/pernya...,"""Pada hari ini kami melakukan pendataan dan me...",demam berdarah
2,"Sabtu, 04 Apr 2020 17:22 WIB",demam berdarah palangka raya,https://news.detik.com/berita/d-4965300/pernya...,Pasien sembuh tercatat bertambah 16 orang. Ang...,demam berdarah
3,"Sabtu, 04 Apr 2020 17:22 WIB",demam berdarah palangka raya,https://news.detik.com/berita/d-4965300/pernya...,"""Kemudian, kita bersyukur bahwa ada 16 saudara...",demam berdarah
4,"Sabtu, 04 Apr 2020 17:22 WIB",demam berdarah palangka raya,https://news.detik.com/berita/d-4965300/pernya...,Berikut ini pernyataan lengkapnya:,demam berdarah
...,...,...,...,...,...
2389,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Sudah ada yang melanjutkan kuliah di bidang p...",tuberkulosis
2390,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"Melihat potensi yang mulai terasah, Esther ber...",tuberkulosis
2391,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Kami ingin (anak-anak di Hohidiai) untuk menj...",tuberkulosis
2392,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Saya merasa banyak bidang sudah pulih dan mer...",tuberkulosis


In [11]:
import pandas as pd

df_location = pd.read_csv('output/df2.csv')

In [12]:
df_1  = pd.read_csv('output/df2.csv')
df_2 = pd.read_csv('output/df3.csv')

In [13]:
df_1['count'] = df_1.groupby(['date','keyword','url','paragraph','category']).cumcount()
df_2['count'] = df_2.groupby(['date','keyword','url','paragraph','category']).cumcount()

df_location = pd.concat([df_1,df_2], ignore_index=False)
df_location = df_location.drop_duplicates()
df_location = df_location.drop(['count'], axis=1)

In [14]:
df

,date,keyword,url,paragraph,category
0,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,Kementerian Kesehatan (Kemenkes) RI mencatat a...,demam berdarah
1,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,"Berdasarkan data Kementerian Kesehatan RI, ber...",demam berdarah
2,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,Ada laporan warga di beberapa daerah menolak j...,demam berdarah
3,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,"""Iya. Jadi semalam sudah. Memang ada spanduk d...",demam berdarah
4,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,Tampaknya di tengah masyarakat ada anggapan ba...,demam berdarah
...,...,...,...,...,...
2389,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Sudah ada yang melanjutkan kuliah di bidang p...",tuberkulosis
2390,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"Melihat potensi yang mulai terasah, Esther ber...",tuberkulosis
2391,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Kami ingin (anak-anak di Hohidiai) untuk menj...",tuberkulosis
2392,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Saya merasa banyak bidang sudah pulih dan mer...",tuberkulosis


In [15]:
df_location.to_csv('output/df.csv')
df_location

,date,keyword,url,paragraph,category
0,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,Kementerian Kesehatan (Kemenkes) RI mencatat a...,demam berdarah
1,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,"Berdasarkan data Kementerian Kesehatan RI, ber...",demam berdarah
2,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,Ada laporan warga di beberapa daerah menolak j...,demam berdarah
3,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,"""Iya. Jadi semalam sudah. Memang ada spanduk d...",demam berdarah
4,NaN,demam berdarah sumatera utara,https://health.detik.com/berita-detikhealth/d-...,Tampaknya di tengah masyarakat ada anggapan ba...,demam berdarah
...,...,...,...,...,...
2389,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Sudah ada yang melanjutkan kuliah di bidang p...",tuberkulosis
2390,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"Melihat potensi yang mulai terasah, Esther ber...",tuberkulosis
2391,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Kami ingin (anak-anak di Hohidiai) untuk menj...",tuberkulosis
2392,"Kamis, 13 Feb 2020 14:17 WIB",tuberkulosis halmahera utara,https://news.detik.com/abc-australia/d-4897770...,"""Saya merasa banyak bidang sudah pulih dan mer...",tuberkulosis
